In [88]:

import requests
from PIL import Image
import pytesseract
from bs4 import BeautifulSoup
import re 
from datetime import datetime, timedelta


In [6]:
# import requests

# # Bước 1: Tải captcha
# def get_captcha():
#     captcha_url = "https://dichvucong.moit.gov.vn/GeneralCaptchaHandler.ashx?iwidth=120&iheight=50&t=638700209522087138"
#     headers = {
#         "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
#         "accept-language": "en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7",
#         "cache-control": "max-age=0",
#         "cookie": "Bifrost_SessionID=qd0ebthlfhstviwckjit3fpp",
#         "dnt": "1",
#         "priority": "u=0, i",
#         "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
#         "sec-ch-ua-mobile": "?0",
#         "sec-ch-ua-platform": '"Windows"',
#         "sec-fetch-dest": "document",
#         "sec-fetch-mode": "navigate",
#         "sec-fetch-site": "none",
#         "sec-fetch-user": "?1",
#         "upgrade-insecure-requests": "1",
#         "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
#     }

#     response = requests.get(captcha_url, headers=headers)
#     if response.status_code == 200:
#         with open("captcha_image.png", "wb") as f:
#             f.write(response.content)
#         print("Captcha đã được lưu vào file captcha_image.png")
#         return True
#     else:
#         print(f"Lỗi khi tải captcha: {response.status_code}")
#         return False

# # Bước 2: Nhập captcha và gửi yêu cầu đăng nhập
# def login_with_captcha():
#     login_url = "https://dichvucong.moit.gov.vn/LoginHandler.ashx"  # URL xử lý đăng nhập (thay thế bằng URL thực tế)
#     headers = {
#         "accept": "application/json, text/javascript, */*; q=0.01",
#         "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
#         "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
#     }

#     # Hiển thị captcha để người dùng nhập
#     captcha_code = input("Nhập mã captcha hiển thị trong file captcha_image.png: ")

#     # Dữ liệu gửi đi
#     data = {
#         "ctl00$cplhContainer$txtLoginName": '2300956022',
#         "ctl00$cplhContainer$txtPassword": '2300956022',
#         "ctl00$cplhContainer$txtCaptcha": captcha_code
#     }

#     response = requests.post(login_url, headers=headers, data=data)
#     if response.status_code == 200:
#         try:
#             data = response.json()  # Chỉ cố gắng parse JSON nếu phản hồi là JSON
#             print("Dữ liệu JSON:", data)
#         except requests.JSONDecodeError:
#             print("Phản hồi không phải JSON. Nội dung phản hồi:")
#             print(response.text)
#     else:
#         print(f"Lỗi HTTP: {response.status_code}")
#         print("Nội dung lỗi:", response.text)

# # Chạy chương trình
# if get_captcha():
#     login_with_captcha()


In [57]:
def request_first_page(fromdate,todate):
    # URL yêu cầu
    url = 'https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx'

    # Đặt các tham số URL (query parameters)
    params = {
        'CertificateDateFrom': fromdate,
        'CertificateDateTo': todate,
        'Status': '-1',
        'FormCOId': '0',
        'CountryId': '0',
        'CertificateNumber': '',
        'CustomsNumber': '',
        'ReceiverName': '',
        'TransportMethodId': '0'
    }

    # Đặt headers
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        'Cookie': 'dn20240109sk2kslshowhideefgdf343ecoDNvbKBCN2dfigzca=yes; Bifrost_SessionID=pvcgdihfrzl20yn414ltvkgk',
        'DNT': '1',
        'Referer': 'https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx?',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"'
    }

    # Gửi yêu cầu GET
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        print("Request thành công!")
        return response.text
    # print(response2.text)  # Nội dung HTML
    else:
        print(f"Request thất bại: {response.status_code}")
        return None


In [56]:
def request_more_page(eventvalidation,viewstate,page_index): 
    import requests

    # URL cần truy vấn
    more_page_url = "https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx"

    # Các headers sử dụng trong request
    headers2 = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Cookie": "dn20240109sk2kslshowhideefgdf343ecoDNvbKBCN2dfigzca=yes; Bifrost_SessionID=pvcgdihfrzl20yn414ltvkgk",
        "DNT": "1",
        "Origin": "https://ecosys.gov.vn",
        "Referer": "https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx?CertificateDateFrom=01/01/2024&CertificateDateTo=31/03/2024&Status=-1&FormCOId=0&CountryId=0&CertificateNumber=&CustomsNumber=&ReceiverName=&TransportMethodId=0",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
    }

    # Dữ liệu form gửi kèm
    data2 = {
        
        "__EVENTVALIDATION": eventvalidation,
        "__VIEWSTATE":viewstate,
        "ctl00$cplhContainer$ddlChoiceIndexOfPage":page_index
    }

    # Gửi POST request
    response2 = requests.post(more_page_url, headers=headers2, data=data2)

    # Kiểm tra kết quả trả về
    if response2.status_code == 200:
        print("Request thành công!")
        return response2.text
        # print(response2.text)  # Nội dung HTML
    else:
        print(f"Request thất bại: {response2.status_code}")
        return None


In [82]:
def generate_dates(start_date, end_date):
    """Tạo danh sách các quý theo từng giai đoạn lùi."""
    quarters = []
    current_start = datetime.strptime(start_date, "%d/%m/%Y")
    current_end = datetime.strptime(end_date, "%d/%m/%Y")
    
    while current_start >= datetime.strptime("01/01/2022", "%d/%m/%Y"):
        quarters.append((current_start.strftime("%d/%m/%Y"), current_end.strftime("%d/%m/%Y")))
        # Lùi lại một quý
        current_end = current_start - timedelta(days=1)
        current_start = (current_end.replace(day=1) - timedelta(days=1)).replace(day=1)
    
    return quarters

# Ngày bắt đầu và kết thúc cho quý hiện tại
start_date = "01/10/2024"
end_date = "31/12/2024"

# Tạo danh sách các quý
dates_list = generate_dates(start_date, end_date)

In [87]:
id_pattern = r"\b\d{8}\b"
ids = []

In [89]:
for start, end in dates_list:
    request_text = request_first_page(start, end)
    if request_text is not None:
        try:
            soup = BeautifulSoup(request_text, "html.parser")
            
            viewstate = soup.find("input", attrs={"type": "hidden", "name": "__VIEWSTATE", "id": "__VIEWSTATE"})["value"]
            eventvalidation = soup.find("input", attrs={"type": "hidden", "name": "__EVENTVALIDATION", "id": "__EVENTVALIDATION"})["value"]
            total_page = int(soup.find("span", attrs={"id": "ctl00_cplhContainer_lblTotalPage"}).string)
            
            if not viewstate or not eventvalidation or not total_page:
                raise ValueError(f"Missing data for request from {start} to {end}. Stopping.")
            
            # print(f"VIEWSTATE: {viewstate}, EVENTVALIDATION: {eventvalidation}, TOTAL_PAGE: {total_page}")
            else: 
                all_td = soup.find_all("td", style="display:none;")
                for td in all_td:
                    if re.fullmatch(id_pattern, td.text.strip()):
                        ids.append(td.text.strip())
                
                for page_index in (2,total_page):
                    request_text_2 = request_more_page(eventvalidation,viewstate,page_index)
                    if request_text_2 is not None:                                  
                        try:
                            soup2 = BeautifulSoup(request_more_page(eventvalidation,viewstate,2),"html.parser")
                            all_td2 = soup.find_all("td", style="display:none;")
                            for td2 in all_td2:
                                if re.fullmatch(id_pattern, td2.text.strip()):
                                    ids.append(td2.text.strip())
                        except Exception as e:
                            print(f"Error processing request from {start} to {end}, page{page_index}: {e}")
                            
        except Exception as e:
            print(f"Error processing request from {start} to {end}: {e}")

    else:
        print(f"Request failed for {start} to {end}.")


Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!
Request thành công!


In [95]:
with open("output_ids.txt", "w", encoding="utf-8") as file:
    for item in ids:
        file.write(f"{item}\n")  # Mỗi ID trên một dòng

In [96]:
len(ids)

720

In [97]:
viewstate

'...'